In [1]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"

In [2]:
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import pipeline
from transformers import AutoImageProcessor, AutoModel
from transformers import CLIPModel, CLIPProcessor
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
import torch
import os
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity 
import pandas as pd

In [3]:
import sys
sys.path.append("geo-image-retrieval") 
import perception_models.core.vision_encoder.pe as pe
import core.vision_encoder.transforms as transforms

In [4]:
# map
with open("../data/database/database_lite.json","r") as f:
    m_idx = json.load(f)
    m_imgs = np.array(m_idx["im_paths"])
    m_loc=np.array(m_idx["loc"])

# query
with open("../data/query/query_lite.json","r") as f:
    q_idx=json.load(f)
    q_imgs=np.array(q_idx["im_paths"])
    q_loc=np.array(q_idx["loc"])
    
# loading the relevance judgements
with h5py.File("../data/london_lite_gt.h5","r") as f:
   fovs = f["fov"][:]
   sim = f["sim"][:].astype(np.uint8)

In [5]:
from huggingface_hub import login
login(new_session=False)

In [6]:
def recall(ranks, pidx, ks):
    recall_at_k = np.zeros(len(ks))
    for qidx in range(ranks.shape[0]):
        for i, k in enumerate(ks):
            if np.sum(np.in1d(ranks[qidx,:k], pidx[qidx])) > 0:
                recall_at_k[i:] += 1
                break

    recall_at_k /= ranks.shape[0]
    return recall_at_k

def apk(pidx, rank, k):
    if len(rank)>k:
        rank = rank[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(rank):
        if p in pidx and p not in rank[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(pidx), k)

def mapk(ranks, pidxs, k):

    return np.mean([apk(a,p,k) for a,p in zip(pidxs, ranks)])

def mapk_many(ranks, pidxs, ks):
    return np.array([mapk(ranks, pidxs, k) for k in ks], dtype=float)

def average_precision(relevant, retrieved):
   precisions = []
   rel = 0
   for i in range(0, len(retrieved)):
      if retrieved[i] in relevant:
         rel += 1
         precisions.append(rel/(i+1))
   return sum(precisions) / len(relevant)

def mean_average_precision(all_relevant, all_retrieved):
   total = 0
   count = 0
   for qid in all_relevant: 
      total += average_precision(all_relevant[qid], all_retrieved.get(qid, []))
      count += 1
   return total / count


def l2_normalize(x, axis=1, eps=1e-12):
   norm = np.linalg.norm(x, axis=axis, keepdims=True)
   return x / (norm + eps)

def get_relevant_images(gt_similarity_matrix, query_idx):
   return np.where(gt_similarity_matrix[query_idx, :] == 1)[0]

def get_retrieved_images(feature_matrix, query_idx):
   return np.argsort(-feature_matrix[query_idx])

def save_results_to_csv(model_name, map_value, recall_at_k, mAPs, csv_path="./results/feature_extraction_evaluation.csv"):
    results_dict = {
        "models_name": model_name,
        "MAP": map_value * 100,
        "Recall@1": recall_at_k[0] * 100,
        "Recall@5": recall_at_k[1] * 100,
        "Recall@10": recall_at_k[2] * 100,
        "Recall@20": recall_at_k[3] * 100,
        "mAP@1": mAPs[0] * 100,
        "mAP@5": mAPs[1] * 100,
        "mAP@10": mAPs[2] * 100,
        "mAP@20": mAPs[3] * 100
    }

    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        if model_name in df['models_name'].values:
            df.loc[df['models_name'] == model_name] = pd.Series(results_dict)
        else:
            df = pd.concat([df, pd.DataFrame([results_dict])], ignore_index=True)
    else:
        df = pd.DataFrame([results_dict])
    
    df.to_csv(csv_path, index=False)
    return df


In [38]:
PERCEPTION_MODELS = {
    'PE-Core-T16-384' : 512,
    'PE-Core-S16-384' : 512, 
    'PE-Core-B16-224' : 1024,
    'PE-Core-L14-336' : 1024, 
    'PE-Core-G14-448' : 1280,   
}

In [ ]:
for MODEL_TO_USE, feat_dim in PERCEPTION_MODELS.items():
    print(f"\n{'='*80}")
    print(f"Loading model: {MODEL_TO_USE}")
    print(f"{'='*80}\n")
    
    try:
        model = pe.VisionTransformer.from_config(MODEL_TO_USE, pretrained=True)
        preprocess = transforms.get_image_transform(model.image_size)
            
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model = model.to(device)
    except Exception as e:
        print(e)
        continue
        
    if os.path.exists(f"./results/pooling_comparison/{MODEL_TO_USE.replace('/', '_')}_pooling_comparison.csv"):
        continue

    m_feats_cls = np.zeros((len(m_imgs), feat_dim), dtype=np.float32)
    
    
    for i, img_name in enumerate(tqdm(m_imgs)):
        img = Image.open(os.path.join('../data/', img_name)).convert("RGB")
        img = preprocess(img).unsqueeze(0).to(device)
        
        with torch.inference_mode():
            vision_outputs = model(img)[0]
        
        m_feats_cls[i] = vision_outputs.squeeze().cpu().numpy()
    
    q_feats_cls = np.zeros((len(q_imgs), feat_dim), dtype=np.float32)
        
    for i, img_name in enumerate(tqdm(q_imgs)):
        img = Image.open(os.path.join('../data/', img_name)).convert("RGB")
        img = preprocess(img).unsqueeze(0).to(device)
        
        with torch.inference_mode():
            vision_outputs = model(img)[0]
        
        q_feats_cls[i] = vision_outputs.squeeze().cpu().numpy()
    
    m_feats_cls = l2_normalize(m_feats_cls, axis=1)
    q_feats_cls = l2_normalize(q_feats_cls, axis=1)
    
    pooling_strategies = {
        "CLS_token": (q_feats_cls, m_feats_cls)
    }
    
    all_results = []
    best = None
    best_map = 0.0
    best_recall_at_k = None
    best_mAPs = None
    
    for pooling_name, (q_feats, m_feats) in pooling_strategies.items():
        print(f"{'='*60}")
        print(f"Evaluating: {pooling_name}")
        print(f"{'='*60}")
        
        similarities = cosine_similarity(q_feats, m_feats)
        
        all_rel = {}
        all_ret = {}
        for query_idx in range(len(similarities)):
            all_rel[query_idx] = get_relevant_images(sim, query_idx)
            all_ret[query_idx] = get_retrieved_images(similarities, query_idx)

        ranks = np.argsort(-similarities, axis=1) 
        
        Q = similarities.shape[0]
        pidx = [np.array(all_rel[q], dtype=int) for q in range(Q)]
        
        ks = [1, 5, 10, 20]
        recall_at_k = recall(ranks, pidx, ks)
        mAPs = mapk_many(ranks, pidx, ks)
        map_value = mean_average_precision(all_rel, all_ret)
        
        if best is None or recall_at_k[0] > best_recall_at_k[0]:
            best_map = map_value
            best = pooling_name
            best_recall_at_k = recall_at_k
            best_mAPs = mAPs
        
        print(f"MODEL: {MODEL_TO_USE}")
        print(f"Pooling: {pooling_name}")
        print(f"MAP: {map_value*100:.2f}%")
        for k, r, m in zip(ks, recall_at_k, mAPs):
            print(f"Recall@{k}: {r*100:.2f}%   mAP@{k}: {m*100:.2f}%")
        print("\n")
        
        result_dict = {
            "models_name": f"{MODEL_TO_USE}_{pooling_name}",
            "MAP": map_value * 100,
            "Recall@1": recall_at_k[0] * 100,
            "Recall@5": recall_at_k[1] * 100,
            "Recall@10": recall_at_k[2] * 100,
            "Recall@20": recall_at_k[3] * 100,
            "mAP@1": mAPs[0] * 100,
            "mAP@5": mAPs[1] * 100,
            "mAP@10": mAPs[2] * 100,
            "mAP@20": mAPs[3] * 100
        }
        all_results.append(result_dict)
    
    df_results = pd.DataFrame(all_results)
    CSV_PATH = f"./results/pooling_comparison/{MODEL_TO_USE.replace('/', '_')}_pooling_comparison.csv"
    os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)
    df_results.to_csv(CSV_PATH, index=False)
    
    df_main = save_results_to_csv(f"{MODEL_TO_USE}_{best}", best_map, best_recall_at_k, best_mAPs)
    
    del model
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    
    print("\n")



Loading model: PE-Core-T16-384

Missing keys for loading vision encoder: []
Unexpected keys for loading vision encoder: []

Loading model: PE-Core-S16-384

Missing keys for loading vision encoder: []
Unexpected keys for loading vision encoder: []


100%|██████████| 500/500 [00:07<00:00, 64.68it/s]
/tmp/ipykernel_6301/2298934414.py:5: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  if np.sum(np.in1d(ranks[qidx,:k], pidx[qidx])) > 0:


Evaluating: CLS_token
MODEL: PE-Core-S16-384
Pooling: CLS_token
MAP: 18.82%
Recall@1: 24.80%   mAP@1: 24.80%
Recall@5: 43.60%   mAP@5: 15.58%
Recall@10: 57.20%   mAP@10: 14.28%
Recall@20: 70.00%   mAP@20: 15.06%





Loading model: PE-Core-B16-224

Missing keys for loading vision encoder: []
Unexpected keys for loading vision encoder: []


100%|██████████| 500/500 [00:06<00:00, 74.52it/s]
/tmp/ipykernel_6301/2298934414.py:5: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  if np.sum(np.in1d(ranks[qidx,:k], pidx[qidx])) > 0:


Evaluating: CLS_token
MODEL: PE-Core-B16-224
Pooling: CLS_token
MAP: 25.13%
Recall@1: 28.80%   mAP@1: 28.80%
Recall@5: 57.80%   mAP@5: 20.77%
Recall@10: 69.20%   mAP@10: 20.18%
Recall@20: 79.60%   mAP@20: 21.39%





Loading model: PE-Core-L14-336

Missing keys for loading vision encoder: []
Unexpected keys for loading vision encoder: []


100%|██████████| 500/500 [00:28<00:00, 17.79it/s]
/tmp/ipykernel_6301/2298934414.py:5: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  if np.sum(np.in1d(ranks[qidx,:k], pidx[qidx])) > 0:


Evaluating: CLS_token
MODEL: PE-Core-L14-336
Pooling: CLS_token
MAP: 23.96%
Recall@1: 31.00%   mAP@1: 31.00%
Recall@5: 57.20%   mAP@5: 20.93%
Recall@10: 65.60%   mAP@10: 19.67%
Recall@20: 76.20%   mAP@20: 20.68%





Loading model: PE-Core-G14-448

Missing keys for loading vision encoder: []
Unexpected keys for loading vision encoder: []


 85%|████████▌ | 426/500 [03:03<00:31,  2.32it/s]]

In [ ]:
df_final = pd.read_csv("./results/feature_extraction_evaluation.csv")
print("\n=== FINAL MODEL COMPARISON (Best pooling for each) ===")
print(df_final[['models_name', 'MAP', 'Recall@1', 'Recall@20']].sort_values('Recall@1', ascending=False).to_string(index=False))